In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import tensorflow as tf
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# Import and Load Data

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

# Data Pre-Processing

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# Run This code to install all the dependencies of nltk natural language Training Kit.
#import nltk
#nltk.download()

In [6]:
#documents
# The job of word tokenizer is a process of splitting a large sample of text into words.
# This is a requirement in natural language processing tasks where each word needs to be 
# captured and subjected to further analysis like classifying

In [7]:
# Words
#documents

# Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [8]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [9]:
# sort classes
classes = sorted(list(set(classes)))

In [34]:
len(classes)
len(words)

88

In [11]:
# documents = combination between patterns and intents
print (len(documents), "documents")

47 documents


In [12]:
# classes = intents
print (len(classes), "classes", classes)

9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [13]:
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [14]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# Training and Testing Data

In [15]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [16]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

In [31]:
bag

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

In [17]:
# create our bag of words array with 1, if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

In [18]:
# output is a '0' for each tag and '1' for current tag (for each pattern)
output_row = list(output_empty)
output_row[classes.index(doc[1])] = 1

In [19]:
output_row

[0, 0, 0, 0, 0, 1, 0, 0, 0]

In [20]:
training.append([bag, output_row])

In [21]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

<ipython-input-21-175f8a2b5529>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [22]:
training

array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 1, 0, 0, 0])]], dtype=object)

In [23]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# Build Model

In [24]:
model = Sequential()
model.add(Dense(64, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [25]:
# Compile model. 
sgd = SGD(lr=0.001, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [26]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
#model.save('chatbot_model.h5', hist)

Epoch 1/200
1/1 [==============================] - 1s 763ms/step - loss: 2.8092 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 7ms/step - loss: 2.2210 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 6ms/step - loss: 2.3657 - accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 6ms/step - loss: 2.7900 - accuracy: 0.0000e+00
Epoch 5/200
1/1 [==============================] - 0s 6ms/step - loss: 2.6471 - accuracy: 0.0000e+00
Epoch 6/200
1/1 [==============================] - 0s 7ms/step - loss: 2.4850 - accuracy: 0.0000e+00
Epoch 7/200
1/1 [==============================] - 0s 6ms/step - loss: 2.4132 - accuracy: 0.0000e+00
Epoch 8/200
1/1 [==============================] - 0s 8ms/step - loss: 2.4806 - accuracy: 0.0000e+00
Epoch 9/200
1/1 [==============================] - 0s 5ms/step - loss: 2.4888 - accuracy: 0.0000e+00
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 2.3795 - accuracy

1/1 [==============================] - 0s 6ms/step - loss: 0.2447 - accuracy: 1.0000
Epoch 84/200
1/1 [==============================] - 0s 6ms/step - loss: 0.1385 - accuracy: 1.0000
Epoch 85/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0674 - accuracy: 1.0000
Epoch 86/200
1/1 [==============================] - 0s 6ms/step - loss: 0.1119 - accuracy: 1.0000
Epoch 87/200
1/1 [==============================] - 0s 6ms/step - loss: 0.1100 - accuracy: 1.0000
Epoch 88/200
1/1 [==============================] - 0s 5ms/step - loss: 0.0288 - accuracy: 1.0000
Epoch 89/200
1/1 [==============================] - 0s 5ms/step - loss: 0.6848 - accuracy: 1.0000
Epoch 90/200
1/1 [==============================] - 0s 6ms/step - loss: 0.2482 - accuracy: 1.0000
Epoch 91/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0532 - accuracy: 1.0000
Epoch 92/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0768 - accuracy: 1.0000
Epoch 93/200
1/1 [===============

1/1 [==============================] - 0s 7ms/step - loss: 0.0113 - accuracy: 1.0000
Epoch 166/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 7ms/step - loss: 0.3180 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 6ms/step - loss: 0.6428 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 5ms/step - loss: 0.1654 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 5ms/step - loss: 0.0213 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 6ms/step - loss: 4.8101e-04 - accuracy: 1.0000
Epoch 175/200
1/1 [=

# Predict responses from our Chatbot

In [27]:
model.save('chatbot_model.h5', hist)

In [28]:
model.predict(train_x)

array([[1.0472011e-03, 3.8274386e-04, 1.0380883e-03, 1.0152130e-03,
        4.7786531e-04, 9.9402738e-01, 5.7038735e-04, 6.7517982e-04,
        7.6595665e-04]], dtype=float32)

In [29]:
len(train_x)

1